In [1]:
#%matplotlib tk
from experiment import *
from dataclasses import replace
import os
import torch
from cugal.config import SinkhornMethod, HungarianMethod

/Users/sebastianrueclausen/Code/cuGAL/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
config = Config(
    sinkhorn_method=SinkhornMethod.LOG,
    hungarian_method=HungarianMethod.SCIPY,
    device="cpu",
    dtype=torch.float32, 
    sinkhorn_cache_size=1,
    sinkhorn_regularization=0.5,
    sinkhorn_iterations=300,
    sinkhorn_threshold=0.0001,
    mu=2.0,
    use_sparse_adjacency=False,
    recompute_distance=False,
    sinkhorn_eval_freq=10,
    frank_wolfe_threshold=None,
    frank_wolfe_iter_count=10,
    iter_count=15,
)

experiment = Experiment(
    graphs=[
        Graph(GraphKind.CA_NETSCIENCE, {
            'n': 128,
            'k': 7,
            'p': 0.1,
        }),
    ],
    algorithms=[
        #Algorithm(replace(config, iter_count = 15), use_fugal=False),
        #Algorithm(replace(config, iter_count = 1), use_fugal=False),
        Algorithm(config, use_fugal=False),
        Algorithm(replace(config, sinkhorn_momentum_start=40), use_fugal=False),
        # Algorithm(replace(config, hungarian_method=HungarianMethod.DENSE), use_fugal=False),
        # Algorithm(replace(config, hungarian_method=HungarianMethod.GREEDY), use_fugal=False),
    ],
    noise_levels=[
        NoiseLevel(0.1, 0.0, False),
    ],
    num_runs=10,
)
results = experiment.run()

folder = "results"
if not os.path.exists(folder): os.makedirs(folder)
# results.dump(folder)\n",


for graph, noise_level, algorithm, result in results.all_results():
    print(result.accuracy)
    print(result.profile.time)
    print([profile.iteration_count for profile in result.profile.sinkhorn_profiles])
    """
    print('graph:', graph)
    print('noise level:', noise_level)
    print('algorithm:', algorithm)
    print()
    print(result)
    print()
    """

Running on CA_NETSCIENCE {'n': 128, 'k': 7, 'p': 0.1}
Running with noise level {'source_noise': 0.1, 'target_noise': 0.0, 'refill_edges': False}
Running with algorithm Cugal {'safe_mode': False, 'device': 'cpu', 'dtype': 'float32', 'sinkhorn_regularization': 0.5, 'sinkhorn_method': 'LOG', 'sinkhorn_iterations': 300, 'sinkhorn_threshold': 0.0001, 'sinkhorn_eval_freq': 10, 'mu': 2.0, 'iter_count': 15, 'frank_wolfe_iter_count': 10, 'frank_wolfe_threshold': None, 'use_sparse_adjacency': False, 'sinkhorn_cache_size': 1, 'sinkhorn_momentum_start': None, 'recompute_distance': False, 'hungarian_method': 'SCIPY'}


λ: 100%|██████████| 15/15 [00:08<00:00,  1.78it/s]


average
Running with algorithm Cugal {'safe_mode': False, 'device': 'cpu', 'dtype': 'float32', 'sinkhorn_regularization': 0.5, 'sinkhorn_method': 'LOG', 'sinkhorn_iterations': 300, 'sinkhorn_threshold': 0.0001, 'sinkhorn_eval_freq': 10, 'mu': 2.0, 'iter_count': 15, 'frank_wolfe_iter_count': 10, 'frank_wolfe_threshold': None, 'use_sparse_adjacency': False, 'sinkhorn_cache_size': 1, 'sinkhorn_momentum_start': 40, 'recompute_distance': False, 'hungarian_method': 'SCIPY'}


λ: 100%|██████████| 15/15 [00:07<00:00,  2.06it/s]


average
0.6242744063324539
7.392257261276245
[300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 242, 242, 242, 242, 242, 242, 242, 242, 242, 190, 186, 186, 186, 186, 186, 184, 184, 184, 184, 136, 126, 126, 126, 126, 126, 126, 126, 126, 126, 27, 13, 11, 13, 13, 13, 13, 11, 11, 13, 25, 11, 11, 11, 11, 11, 11, 11, 11, 11, 25, 11, 11, 11, 11, 11, 11, 11, 11, 11, 25, 11, 11, 11, 11, 11, 11, 11, 11, 11, 25, 11, 11, 11, 11, 11, 11, 11, 11, 11, 31, 11, 11, 11, 11, 11, 11, 11, 11, 11, 25, 11, 11, 11, 11, 11, 11, 11, 11, 11]
0.6269129287598945
6.257170581817627
[300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 242, 242, 300, 2

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker


matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'font.size' : 7,
    'text.usetex': True,
    'pgf.rcfonts': False,
})

# Average two runs.
standard = (np.array([300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 246, 244, 242, 242, 242, 242, 242, 242, 242, 242, 188, 242, 242, 242, 242, 242, 184, 242, 224, 242, 132, 126, 126, 126, 126, 126, 126, 126, 126, 126, 76, 68, 68, 68, 68, 68, 68, 68, 68, 68, 23, 11, 11, 11, 11, 11, 11, 11, 11, 11, 21, 11, 11, 11, 11, 11, 11, 11, 11, 11, 21, 11, 11, 11, 11, 11, 11, 11, 11, 11, 21, 11, 11, 11, 11, 11, 11, 11, 11, 11, 21, 11, 11, 11, 11, 11, 11, 11, 11, 11, 21, 11, 11, 11, 11, 11, 11, 11, 11, 11])
    + np.array([300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 275, 274, 274, 273, 273, 273, 273, 272, 272, 246, 244, 243, 242, 242, 242, 243, 271, 271, 242, 192, 213, 213, 213, 185, 184, 213, 184, 185, 184, 133, 126, 126, 126, 126, 126, 126, 126, 126, 126, 107, 97, 97, 97, 97, 97, 97, 97, 97, 97, 79, 68, 68, 68, 68, 68, 68, 68, 68, 68, 52, 39, 39, 39, 39, 39, 39, 39, 39, 39, 25, 11, 11, 11, 11, 11, 11, 11, 11, 11, 25, 11, 11, 11, 11, 11, 11, 11, 11, 11, 26, 11, 11, 11, 11, 11, 11, 11, 11, 11])) / 2
accel = (np.array([300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 246, 246, 300, 244, 242, 300, 242, 188, 184, 184, 184, 184, 74, 128, 70, 68, 68, 132, 126, 68, 13, 13, 68, 11, 11, 11, 11, 132, 126, 126, 11, 11, 11, 11, 68, 11, 11, 76, 11, 11, 11, 11, 15, 15, 11, 11, 11, 23, 11, 11, 11, 11, 11, 11, 11, 11, 11, 23, 11, 11, 11, 11, 11, 11, 11, 11, 11, 23, 11, 11, 11, 11, 11, 11, 11, 11, 11, 23, 11, 11, 11, 11, 11, 11, 11, 11, 11, 23, 11, 11, 11, 11, 11, 11, 11, 11, 11, 23, 11, 11, 11, 11, 11, 11, 11, 11, 11])
    + np.array([300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 271, 244, 186, 187, 185, 184, 126, 126, 126, 155, 68, 108, 97, 97, 97, 68, 68, 68, 68, 68, 68, 80, 68, 68, 40, 40, 12, 12, 12, 11, 11, 52, 68, 39, 39, 39, 11, 12, 11, 12, 12, 53, 39, 39, 11, 11, 11, 11, 11, 11, 12, 25, 11, 11, 11, 11, 11, 11, 11, 11, 11, 25, 11, 11, 11, 11, 11, 11, 11, 11, 11, 25, 11, 11, 11, 11, 11, 11, 11, 11, 11, 25, 11, 11, 11, 11, 11, 11, 11, 11, 11, 26, 11, 11, 11, 11, 11, 11, 11, 11, 11])) / 2

print(np.mean(standard), np.mean(accel))

# Hack to align.
x = np.linspace(-0.5, 149.5, 150)

for l in range(0, 150, 10):
    plt.axvline(x = l, linestyle='--', alpha=0.5, linewidth=1)

plt.plot(x, standard, label='No Acceleration')
plt.plot(x, accel, label='Acceleration')

plt.xlabel('$\lambda$')
plt.ylabel('Sinkhorn-Knopp iterations')

def format_x_axis(x, pos):
    return f'{int(x//10)}'
formatter = ticker.FuncFormatter(format_x_axis)
plt.gca().xaxis.set_major_formatter(formatter)

plt.legend()

# Show the plot
# plt.show()
plt.savefig('figure.pgf')

161.05666666666667 122.64333333333333


<>:33: SyntaxWarning: invalid escape sequence '\l'
<>:33: SyntaxWarning: invalid escape sequence '\l'
/var/folders/j1/g9r2fg9n15b7tl6rq981h9y40000gn/T/ipykernel_9802/2699971752.py:33: SyntaxWarning: invalid escape sequence '\l'
  plt.xlabel('$\lambda$')
